In [2]:
# -*- coding: utf-8 -*-
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import ht
import CoolProp as cp
import fluids

### INTERCAMBIADORES DE CALOR


#### OBJETIVO

Diseño de un intercambiador de tipo casco y tubos para el calentamiento de etilenglicol, procesado en una industria química. 

#### ENUNCIADO

Se requieren calentar un cantidad $\dot m_F$ de etilenglicol desde un temperatura $T_{FE}$, garantizando a la salida una temperatura  $T_{FS}=40^\circ$ C. Para ello se dispone de un caudal $\dot m_C$ de agua que ingresa a $T_{CE}=65^\circ$C. El agua no podrá enfriarse por debajo de los $T_{CS}=20^\circ$ C por requerimientos del sistema de calentamiento de la planta. A los efectos del proceso, no afecta que bajo situaciones especiales la temperatura del etilenglicol esté por encima de lo establecido. Considerar al agua como el fluido que pasa por los tubos y al etilenglicol en un por la carcasa.
 

Se pide:

    1. Determinar las características de un intercambiador de casco y tubos que cumpla con los requerimientos. 
    2. Calcular las caídas de presión de ambas corrientes en su paso por el equipo.
    3. Completar el cuadro de resumen que contenga las características constructivas y de operación fundamentales calculadas (Entregar el archivo Excel completo adjunto).
    4. Hacer un plano conceptual a escala del equipo.
    5. Hacer un plano a escala de la placa portatubos y su relación con la carcasa.
    6. Durante la operación, el proceso sufre cierto desbalance que hace que la temperatura de entrada del etilenglicol disminuya 5ºC. Manteniendo los caudales constantes, determinar las nuevas temperaturas de salida. ¿Verifican las condiciones de funcionamiento requeridas? (Se recomienda utilizar el método e-NUT)
    7. En los casos que no verifiquen, se dispone de un sistema de control del caudal de agua, para poder llevar al equipo dentro de los parámetros de funcionamiento adecuados. ¿Cuál es este nuevo caudal? (Se recomienda utilizar el método e-NUT)


DATOS

Corriente de Servicio: Agua de caldera tratada.

Corriente de Proceso: Etilenglicol

Temperatura del etilenglicol a la entrada – TFE [ºC] = 5+0.5·UNP

Temperatura del etilenglicol a la salida – TFS [ºC] = 40

Temperatura del agua a la entrada – TCE [ºC] = 65

Temperatura del agua de salida mínima – TFSmin [ºC] = 20

Caudal de etilenglicol – mF [kg/s] (45+UNP)/10

Nota:
    • UNP= último número de padrón (alumnos de intercambio UNP=4)






In [10]:
################
#     DATOS    #
################


Tfe = 25
Tfs_max = 40
Tce = 72
Tcs = 50
Tfs  = Tfs_max
m_c = 4.9
L_tubos = 1.5
Rf = 2e-4

#Calor intercambiado


#Q = C_c*m_c*(Tcs-Tce)
Tc_media0 = (Tce+Tcs)/2
C_c = cp.CoolProp.PropsSI('C','T',273+Tc_media0,'P',101.325e3,'Water')
mu_c = cp.CoolProp.PropsSI('V','T',273+Tc_media0,'P',101.325e3,'Water')
Q = C_c*m_c*(Tce-Tcs)


Tf_media = (Tfe+Tfs)/2
C_f = cp.CoolProp.PropsSI('C','T',273+Tf_media,'P',101.325e3,'INCOMP::MEG-20%')
m_f = Q / C_f/(Tfs-Tfe)




Cmin = np.min([C_c*m_c,C_f*m_f])
Cmax = np.max([C_c*m_c,C_f*m_f])

Cr = Cmin/Cmax

Qmax = Cmin*(Tce-Tfe) 
epsilon = Q/Qmax

dTlm = ht.LMTD(Tci=Tfe, Tco=Tfs, Thi=Tce, Tho=Tcs)

# dos pasos de carcaza
Ft = ht.F_LMTD_Fakheri(Tci=Tfe, Tco=Tfs, Thi=Tce, Tho=Tcs, shells=1)
UA = Q/(dTlm*Ft)
#Nut = ht.NTU_from_effectiveness(effectiveness=epsilon, Cr=Cr, subtype='S&T')
Nut = ht.NTU_from_UA(UA,Cmin)

print('m_f = %.2f'%m_f)
print('DTLM = %.1f '%dTlm)
print('Ft = %.2f '%Ft)
print('UA = %.2f '%UA)
print('Nut =%.2f '%Nut)


m_f = 7.68
DTLM = 28.4 
Ft = 0.93 
UA = 17161.50 
Nut =0.84 


In [11]:
#U dentro de [800,1600]
U = 700
A = UA/U

D = 25.4e-3
e = 9e-4
Di = D-2*e

Ntubos_0  = np.int(A / (np.pi*D*L_tubos))

print('Area propuesta = %.2f'%A)

Area propuesta = 24.52


/tmp/ipykernel_21476/3096128654.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Ntubos_0  = np.int(A / (np.pi*D*L_tubos))


In [12]:
#Proponemos Re~10000
Re_design = 0.7e4
m_t = m_c  # fluido caliente por tubos
#Red = 4*m_t*Npasotubos / (np.pi*D*Ntubos )
Npasotubos =  Re_design*(np.pi*Di*Ntubos_0*mu_c ) / (4*m_t)

Npasotubos = np.ceil(Npasotubos)+1

print('N tubos = %0d    N paso tubos = %0d'%(Ntubos_0,Npasotubos))


N tubos = 204    N paso tubos = 4


In [13]:
Dcarcasa2 = ht.DBundle_for_Ntubes_Phadkeb(Ntubos_0, D, 1.25*25e-3, Npasotubos, angle=90)

print('Diametro Carcasa = %.2f\'\''%(Dcarcasa2/25.4e-3))

Dcarcasa = 23.25*25.4e-3 ##el valor mas cercano a Dcarcasa2
Do = D
pitch = 1.25*25.4e-3


Ntubos = ht.Ntubes(Dcarcasa, D, pitch, Ntp=Npasotubos, angle=90, Method=None)

print('Diametro Carcasa final = %.2f\'\''%(Dcarcasa/25.4e-3))
print('Numero de tubos = %0d'%(Ntubos))

Diametro Carcasa = 22.17''
Diametro Carcasa final = 23.25''
Numero de tubos = 208


Arreglo cuadrado dX= 1.25'' D=1'' carcasa 23.25''
determina Ntubos en su interior. Se va probando parámetros para ajustar el valor inicial
Ntubos_0

In [14]:
A_real = np.pi*L_tubos*D*Ntubos
print('Areal real = %.2f m2'%A_real)

Areal real = 24.90 m2


In [15]:
Re_d = 4*m_t*Npasotubos / (np.pi*D*Ntubos*mu_c )
print('Reynolds = %.f'%Re_d)

Pr_c = cp.CoolProp.PropsSI('PRANDTL','T',273+Tc_media0,'P',101.325e3,'Water')
k_c = cp.CoolProp.PropsSI('L','T',273+Tc_media0,'P',101.325e3,'Water')

Reynolds = 10266


In [16]:
Nu_c = ht.conv_internal.turbulent_Colburn(Re_d,Pr_c)
h_c = Nu_c*k_c/Di

In [17]:
# En la carcasa

B = 0.25*Dcarcasa

#Pag 76 Cao, Dequivalente
Pt = pitch
c = (1.25-1)*25.4e-3
As = Dcarcasa*c*B/Pt
Gs = m_f / As


Deq = 4*(pitch**2-np.pi*D**2/4) / (np.pi*D)
Aeq =  Deq**2*np.pi/4*Ntubos*Npasotubos/4

Pr_f = cp.CoolProp.PropsSI('PRANDTL','T',273+Tf_media,'P',101.325e3,'INCOMP::MEG-20%')
k_f = cp.CoolProp.PropsSI('L','T',273+Tf_media,'P',101.325e3,'INCOMP::MEG-20%')
mu_f = cp.CoolProp.PropsSI('V','T',273+Tf_media,'P',101.325e3,'INCOMP::MEG-20%')

Re_f = Deq*m_f/mu_f/Aeq

Re_f = Deq*Gs/mu_f
Nu_f = 0.36*Re_f**0.55*Pr_f**0.3333

h_f = Nu_f*k_f/Deq

print('hf = %.2f   hc=%.2f'%(h_f,h_c))

hf = 2353.05   hc=1475.32


In [18]:
# Temperatura de pared Tw

# (Tcm-Tw)*A*h_c = (Tw-Tfm) *A*h_f
#Tw*h_c+Tw*h_f = Tcm*h_c + Tfm*h_f

Tw = (h_c*Tc_media0+h_f*Tf_media) / (h_c+h_f )
#con este valor se corrigen las propiedades de los fluidos

Tcmedia = (Tc_media0+Tw)/2
Tfmedia = (Tf_media+Tw)/2

Pr_f = cp.CoolProp.PropsSI('PRANDTL','T',273+Tfmedia,'P',101.325e3,'INCOMP::MEG-20%')
k_f = cp.CoolProp.PropsSI('L','T',273+Tfmedia,'P',101.325e3,'INCOMP::MEG-20%')
mu_f = cp.CoolProp.PropsSI('V','T',273+Tfmedia,'P',101.325e3,'INCOMP::MEG-20%')

Pr_c = cp.CoolProp.PropsSI('PRANDTL','T',273+Tcmedia,'P',101.325e3,'Water')
k_c = cp.CoolProp.PropsSI('L','T',273+Tcmedia,'P',101.325e3,'Water')
mu_c = cp.CoolProp.PropsSI('V','T',273+Tcmedia,'P',101.325e3,'Water')
rho_c = cp.CoolProp.PropsSI('D','T',273+Tcmedia,'P',101.325e3,'Water')
rho_f = cp.CoolProp.PropsSI('D','T',273+Tfmedia,'P',101.325e3,'Water')

mu_w = cp.CoolProp.PropsSI('V','T',273+Tw,'P',101.325e3,'Water')


In [19]:
Re_f = Deq*m_f/mu_f/Aeq

Re_f = Deq*Gs/mu_f
Nu_f = 0.36*Re_f**0.55*Pr_f**0.3333

h_f = Nu_f*k_f/Deq

Nu_c = ht.conv_internal.turbulent_Colburn(Re_d,Pr_c)
h_c = Nu_c*k_c/Di
print('hf = %.2f   hc=%.2f'%(h_f,h_c))

hf = 2438.14   hc=1529.99


In [20]:
Ulimpio = 1 / (D/Di/h_c+1/h_f )
U = 1 / (1/Ulimpio + Rf *(1+D/Di))



A_calculo = Q / (U*dTlm*Ft)

print('U=%.2f   Acalculo = %.2f'%(U,A_calculo))
print('A real inicial %.2f'%A_real)

U=654.08   Acalculo = 26.24
A real inicial 24.90


In [21]:
# Caida de presion

# Calculo para flujo en tubos
roughness = fluids.roughness_Farshad('Carbon steel, bare', D=D)

f_c = fluids.friction.friction_factor(Re=Re_d, eD=roughness/Di)

f_c2 = 1.2*(0.0014+0.125/Re_d**.32)

Gtubos = m_c*Npasotubos*4 / (Ntubos*np.pi*Di**2)


Delta_p_friccion = 4*f_c*Npasotubos*L_tubos*Gtubos**2/(2*Di*rho_c)*(mu_c/mu_w)**-0.14

Delta_p_cabezales = 4*Npasotubos*Gtubos**2/rho_c/2

Delta_p_tubos = Delta_p_cabezales+Delta_p_friccion

print('factor friccion tubos = %4f '%f_c)
print('Gtubos = %.1f'%Gtubos)
print('Delta p friccion = %.2f'%Delta_p_friccion)
print('Delta p cabezales = %.2f'%Delta_p_cabezales)
print('Delta p tubos = %.2f'%Delta_p_tubos)

factor friccion tubos = 0.032919 
Gtubos = 215.4
Delta p friccion = 803.71
Delta p cabezales = 376.10
Delta p tubos = 1179.81


In [22]:
#calculo para flujo en carcasa


LSpacing = Dcarcasa*0.25
NBaffles = np.floor(L_tubos / LSpacing)

Delta_p_carcasa = ht.conv_tube_bank.dP_Kern(m_f, rho_f, mu_f, Dcarcasa, LSpacing, pitch, D, NBaffles, mu_w=mu_w)
print('Delta p carcasa = %.1f'%Delta_p_carcasa)


Delta p carcasa = 7687.7
